In [1]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [2]:
# Path dataset di Google Drive (ubah sesuai lokasi file ZIP Anda)
drive_zip_path = '/content/drive/MyDrive/Dataset/Corn or Maize Leaf Disease.zip'

# Path tujuan di Colab
colab_extract_path = '/content/dataset'

# Ekstrak dataset
import zipfile
import os

with zipfile.ZipFile(drive_zip_path, 'r') as zip_ref:
    zip_ref.extractall(colab_extract_path)

# Periksa isi folder setelah ekstraksi
print("Folder dataset setelah ekstraksi:", os.listdir(colab_extract_path))


Folder dataset setelah ekstraksi: ['data']


In [5]:
# Cek struktur dataset setelah ekstraksi
for root, dirs, files in os.walk(colab_extract_path):
    print(root, "->", len(files), "files")

/content/dataset -> 0 files
/content/dataset/data -> 0 files
/content/dataset/data/Common_Rust -> 1306 files
/content/dataset/data/Blight -> 1146 files
/content/dataset/data/Healthy -> 1162 files
/content/dataset/data/Gray_Leaf_Spot -> 574 files


In [7]:
# Path ke folder data
data_dir = os.path.join(colab_extract_path, 'data')

# Folder target untuk train dan test
train_dir = '/content/dataset/train'
test_dir = '/content/dataset/test'

os.makedirs(train_dir, exist_ok=True)
os.makedirs(test_dir, exist_ok=True)

# Ambil semua folder kelas di dalam 'data'
classes = [d for d in os.listdir(data_dir) if os.path.isdir(os.path.join(data_dir, d))]

# Pindahkan file ke folder train dan test
for class_name in classes:
    class_path = os.path.join(data_dir, class_name)
    images = os.listdir(class_path)

    # Bagi data menjadi train dan test
    from sklearn.model_selection import train_test_split
    train_images, test_images = train_test_split(images, test_size=0.2, random_state=42)

    # Buat subfolder untuk setiap kelas di train dan test
    os.makedirs(os.path.join(train_dir, class_name), exist_ok=True)
    os.makedirs(os.path.join(test_dir, class_name), exist_ok=True)

    # Pindahkan data ke folder train dan test
    for img in train_images:
        shutil.move(os.path.join(class_path, img), os.path.join(train_dir, class_name, img))
    for img in test_images:
        shutil.move(os.path.join(class_path, img), os.path.join(test_dir, class_name, img))


In [8]:
print("Isi folder train:", os.listdir(train_dir))
print("Isi folder test:", os.listdir(test_dir))


Isi folder train: ['Common_Rust', 'Blight', 'Healthy', 'Gray_Leaf_Spot']
Isi folder test: ['Common_Rust', 'Blight', 'Healthy', 'Gray_Leaf_Spot']


In [9]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(
    rescale=1.0/255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True
)
test_datagen = ImageDataGenerator(rescale=1.0/255)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(150, 150),
    batch_size=32,
    class_mode='categorical'
)

test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(150, 150),
    batch_size=32,
    class_mode='categorical'
)


Found 3348 images belonging to 4 classes.
Found 840 images belonging to 4 classes.


In [10]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout

# Membangun model
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(150, 150, 3)),
    MaxPooling2D(pool_size=(2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),
    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(len(train_generator.class_indices), activation='softmax')  # Jumlah output sesuai jumlah kelas
])

# Kompilasi model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Menampilkan ringkasan model
model.summary()


/usr/local/lib/python3.10/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                      │ (None, 148, 148, 32)        │             896 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d (MaxPooling2D)         │ (None, 74, 74, 32)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_1 (Conv2D)                    │ (None, 72, 72, 64)          │          18,496 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_1 (MaxPooling2D)       │ (None, 36, 36, 64)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_2 (Conv2D)                    │ (None, 34, 34, 128)         │          73,856 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_2 (MaxPooling2D)       │ (None, 17, 17, 128)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten (Flatten)                    │ (None, 36992)               │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 128)                 │       4,735,104 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ (None, 128)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 4)                   │             516 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 4,828,868 (18.42 MB)

 Trainable params: 4,828,868 (18.42 MB)

 Non-trainable params: 0 (0.00 B)

In [11]:
# Latih model
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // train_generator.batch_size,
    epochs=10,  # Anda bisa meningkatkan jumlah epoch
    validation_data=test_generator,
    validation_steps=test_generator.samples // test_generator.batch_size
)


Epoch 1/10


/usr/local/lib/python3.10/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


104/104 ━━━━━━━━━━━━━━━━━━━━ 207s 2s/step - accuracy: 0.6137 - loss: 0.8893 - val_accuracy: 0.8077 - val_loss: 0.5523
Epoch 2/10
104/104 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.9375 - loss: 0.4817 - val_accuracy: 0.8750 - val_loss: 0.3750
Epoch 3/10


/usr/lib/python3.10/contextlib.py:153: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(typ, value, traceback)


104/104 ━━━━━━━━━━━━━━━━━━━━ 261s 2s/step - accuracy: 0.8005 - loss: 0.4952 - val_accuracy: 0.8353 - val_loss: 0.4110
Epoch 4/10
104/104 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.8438 - loss: 0.3648 - val_accuracy: 0.7500 - val_loss: 0.3311
Epoch 5/10
104/104 ━━━━━━━━━━━━━━━━━━━━ 255s 2s/step - accuracy: 0.8300 - loss: 0.4199 - val_accuracy: 0.8257 - val_loss: 0.3978
Epoch 6/10
104/104 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.8125 - loss: 0.4901 - val_accuracy: 0.8750 - val_loss: 0.2500
Epoch 7/10
104/104 ━━━━━━━━━━━━━━━━━━━━ 203s 2s/step - accuracy: 0.8118 - loss: 0.4426 - val_accuracy: 0.8305 - val_loss: 0.3445
Epoch 8/10
104/104 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.8750 - loss: 0.3141 - val_accuracy: 1.0000 - val_loss: 0.2068
Epoch 9/10
104/104 ━━━━━━━━━━━━━━━━━━━━ 200s 2s/step - accuracy: 0.8340 - loss: 0.3922 - val_accuracy: 0.8534 - val_loss: 0.3394
Epoch 10/10
104/104 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.8125 - loss: 0.3958 - val_accuracy: 0.7500 -

In [12]:
# Evaluasi model
test_loss, test_acc = model.evaluate(test_generator)
print(f"Test Loss: {test_loss}, Test Accuracy: {test_acc}")


27/27 ━━━━━━━━━━━━━━━━━━━━ 14s 490ms/step - accuracy: 0.8521 - loss: 0.3374
Test Loss: 0.3363024890422821, Test Accuracy: 0.851190447807312


In [13]:
# Simpan model
model_path = '/content/drive/MyDrive/leaf_disease_model.h5'
model.save(model_path)
print(f"Model disimpan di {model_path}")


Model disimpan di /content/drive/MyDrive/leaf_disease_model.h5


In [14]:
!pip install tensorflowjs

!tensorflowjs_converter --input_format keras \
    /content/drive/MyDrive/leaf_disease_model.h5 \
    /content/drive/MyDrive/tfjs_model


INFO: pip is looking at multiple versions of tf-keras to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.1/89.1 kB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.9/15.9 MB 60.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 615.3/615.3 MB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 46.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 68.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.3/9.3 MB 58.0 MB/s eta 0:00:00
  Attempting uninstall: packaging
    Found existing installation: packaging 24.2
    Uninstalling packaging-24.2:
      Successfully uninstalled packaging-24.2
  Attempting uninstall: tensorboard
    Found existing installation: tensorboard 2.17.1
    Uninstalling tensorboard-2.17.1:
      Successfully unin

2024-11-20 09:21:16.671713: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1732094476.698696   11973 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1732094476.706847   11973 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
🌲 Try https://ydf.readthedocs.io, the successor of TensorFlow Decision Forests with more features and faster training!
failed to lookup keras version from the file,
    this is likely a weight only file
